<a href="https://colab.research.google.com/github/AmiriHayes/sandbox/blob/main/august/homophily_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PART 1 / 4:
 SYNTHETIC DATASET AND FEATURE ENGINEERING

In [ ]:
# -------------------------
# all imports for notebook:
# -------------------------

!pip install torch_geometric

import os
import csv
import requests
import numpy as np
import pandas as pd
import torch
import networkx as nx
from tqdm import tqdm
from google.colab import files
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [11]:
# -------------------------------------------------
# save all code in data folder to personal computer
# -------------------------------------------------

def save_all_data():
    !zip -r /content/saved_file.zip /content/data
    files.download("/content/saved_file.zip")

In [2]:
# --------------------------------------------------------------------
# simulate dataset w/ avg node homophily value and max_num_neighbors K
# --------------------------------------------------------------------

n_nodes = 10_000
n_classes = 7
p_same = 0.8
K = 100
seed = 42

rng = np.random.default_rng(seed)
OUT_GRAPH_DIR = "data/simulated_graph"
OUT_FEATURE_DIR = "data/simulated_features"
os.makedirs(OUT_GRAPH_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)

labels = rng.integers(low=0, high=n_classes, size=n_nodes)
nodes_df = pd.DataFrame({"nodeId": np.arange(n_nodes), "label": labels})
nodes_df.to_csv(os.path.join(OUT_GRAPH_DIR, "simulated_nodes.csv"), index=False)

class_nodes = {c: np.where(labels == c)[0].tolist() for c in range(n_classes)}
all_nodes = np.arange(n_nodes).tolist()

def sample_from_pool(pool, m, excludes=set(), rng=None):
    pool_set = set(pool) - excludes
    pool_list = list(pool_set)
    if m == 0: return []
    if len(pool_list) == 0: return list(rng.choice(pool, size=m, replace=True))
    if len(pool_list) >= m: return list(rng.choice(pool_list, size=m, replace=False))
    else:
        result = pool_list.copy()
        rem = m - len(result)
        result += list(rng.choice(pool_list, size=rem, replace=True))
        return result

edges_set = set()
for i in tqdm(range(n_nodes), desc="constructing edges"):
    k = int(rng.integers(0, K+1))  # uniform integer in [0, K]
    if k == 0: continue
    n_same = int(rng.binomial(k, p_same))
    n_diff = k - n_same

    same_pool = class_nodes[labels[i]]
    excludes = {i}

    same_choices = []
    if n_same > 0:
        same_choices = sample_from_pool(same_pool, n_same, excludes=excludes, rng=rng)
        excludes = excludes.union(same_choices)

    other_pool = [x for x in all_nodes if labels[x] != labels[i]]
    diff_choices = []
    if n_diff > 0:
        diff_choices = sample_from_pool(other_pool, n_diff, excludes=excludes, rng=rng)

    for j in same_choices + diff_choices:
        if j == i: continue
        a, b = (i, j) if i < j else (j, i)
        edges_set.add((a, b))

edges = sorted(list(edges_set))
edges_df = pd.DataFrame(edges, columns=["sourceNodeId", "targetNodeId"])
edges_df.to_csv(os.path.join(OUT_GRAPH_DIR, "simulated_edges.csv"), index=False)

G = nx.Graph()
G.add_nodes_from(range(n_nodes))
G.add_edges_from(edges)

homophily = []
for node in range(n_nodes):
    nbrs = list(G.neighbors(node))
    deg = len(nbrs)
    if deg == 0:
        homophily.append(0.0)
    else:
        same_count = sum(1 for nb in nbrs if labels[nb] == labels[node])
        homophily.append(same_count / deg)

homophily_df = pd.DataFrame({"nodeId": np.arange(n_nodes), "homophily": homophily})
homophily_df.to_csv(os.path.join(OUT_GRAPH_DIR, "simulated_homophily.csv"), index=False)

print("\nSanity Check:")
print(f"Generated graph: n_nodes={n_nodes}, n_edges={G.number_of_edges()}")
print(f"Homophily Mean: {np.mean(homophily):.4f}, Std: {np.std(homophily):.4f}")

degree_dict = dict(G.degree())

triangles = nx.triangles(G)
triangle_density = {}
for node in G.nodes():
    d = degree_dict[node]
    if d >= 2:
        possible = d * (d - 1) / 2
        triangle_density[node] = triangles[node] / possible
    else: triangle_density[node] = 0.0

avg_neighbor_degree = nx.average_neighbor_degree(G)

try:
    eigen_centrality = nx.eigenvector_centrality_numpy(G)
except Exception:
    eigen_centrality = nx.eigenvector_centrality(G, max_iter=1000)

def save_feature_series(values_dict, path):
    s = pd.Series([values_dict[i] for i in range(n_nodes)])
    s.to_csv(path, index=False, header=False)

save_feature_series(degree_dict, os.path.join(OUT_FEATURE_DIR, "node_degree.csv"))
save_feature_series(triangle_density, os.path.join(OUT_FEATURE_DIR, "local_triangle_density.csv"))
save_feature_series(avg_neighbor_degree, os.path.join(OUT_FEATURE_DIR, "average_neighbor_degree.csv"))
save_feature_series(eigen_centrality, os.path.join(OUT_FEATURE_DIR, "eigenvector_centrality.csv"))

print("\nSaved features to:", OUT_FEATURE_DIR)
print("Saved graph files to:", OUT_GRAPH_DIR)

constructing edges: 100%|██████████| 10000/10000 [00:55<00:00, 178.94it/s]



Sanity Check:
Generated graph: n_nodes=10000, n_edges=493302
Homophily Mean: 0.7987, Std: 0.0424

Saved features to: data/simulated_features
Saved graph files to: data/simulated_graph


### PART 2 / 4:
REGRESSION ON THE SYNTHETIC DATA W/ LINEAR REGRESSION & GNNs

In [9]:
# --------------------------------------------------------
# linear regression implementation w/ synthetic graph data
# --------------------------------------------------------
FEATURE_DIR = "data/simulated_features"
GRAPH_DIR = "data/simulated_graph"

feature_files = [
    ("node_degree", "node_degree.csv"),
    ("local_triangle_density", "local_triangle_density.csv"),
    ("average_neighbor_degree", "average_neighbor_degree.csv"),
    ("eigenvector_centrality", "eigenvector_centrality.csv"),
]

X_df = pd.DataFrame()
for fname, ffile in feature_files:
    path = os.path.join(FEATURE_DIR, ffile)
    vals = pd.read_csv(path, header=None).squeeze("columns")
    X_df[fname] = vals

num_nodes = 10_000
X_df['node_degree'] = np.random.rand(num_nodes)
X_df['local_triangle_density'] = np.random.rand(num_nodes)
X_df['average_neighbor_degree'] = np.random.rand(num_nodes)
X_df['eigenvector_centrality'] = np.random.rand(num_nodes)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_df)
X_df = pd.DataFrame(X_scaled, columns=X_df.columns)

y_path = os.path.join(GRAPH_DIR, "simulated_homophily.csv")
y_df = pd.read_csv(y_path)["homophily"]

assert len(X_df) == len(y_df), "Feature/target length mismatch!"
print("Number of nodes:", len(X_df))
print("Any NaNs in features?", X_df.isna().any().any())
print("Any NaNs in target?", y_df.isna().any())

print("\nFeature summary stats:")
print(X_df.describe())

X_train, X_test, y_train, y_test = train_test_split(
    X_df, y_df, test_size=0.2, random_state=42
)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2 = r2_score(y_test, y_pred)

print(f"\nTest MSE: {mse:.4f}")
print(f"Test R²: {r2:.4f}")
print(f"Mean Percentage Errror: {mape:.2f}%")

coef_df = pd.DataFrame({
    "feature": X_df.columns,
    "coefficient": reg.coef_,
    "abs_coeff": np.abs(reg.coef_)
}).sort_values(by="abs_coeff", ascending=False)

print("\nFeature importance (by absolute coefficient value):")
print(coef_df[["feature", "coefficient"]])

pred_check = pd.DataFrame({
    "actual": y_test.values[:10],
    "predicted": y_pred[:10]
})
print("\nSample predictions vs. actual:")
print(pred_check)

Number of nodes: 10000
Any NaNs in features? False
Any NaNs in target? False

Feature summary stats:
        node_degree  local_triangle_density  average_neighbor_degree  \
count  1.000000e+04            1.000000e+04             1.000000e+04   
mean   5.924150e-17           -2.149392e-17             2.657430e-16   
std    1.000050e+00            1.000050e+00             1.000050e+00   
min   -1.716156e+00           -1.735200e+00            -1.738337e+00   
25%   -8.753201e-01           -8.703405e-01            -8.588104e-01   
50%   -8.156145e-03            1.272916e-02             1.967068e-03   
75%    8.781689e-01            8.527567e-01             8.619247e-01   
max    1.727453e+00            1.737419e+00             1.731724e+00   

       eigenvector_centrality  
count            1.000000e+04  
mean             1.413980e-16  
std              1.000050e+00  
min             -1.735987e+00  
25%             -8.573671e-01  
50%             -3.118730e-03  
75%              8.559078e

In [22]:
# --------------------------------------------------------------------
# graph convolutional network implementation with synthetic graph data
# --------------------------------------------------------------------

nodes_df = pd.read_csv("data/simulated_graph/simulated_nodes.csv") # header - nodeId,label
edges_df = pd.read_csv("data/simulated_graph/simulated_edges.csv") # header - sourceNodeId,targetNodeId
homophily_df = pd.read_csv("data/simulated_graph/simulated_homophily.csv") # header - nodeId, homophily

G = nx.Graph()
G.add_nodes_from(nodes_df['nodeId'])
G.add_edges_from(zip(edges_df['sourceNodeId'], edges_df['targetNodeId']))

num_nodes = nodes_df.shape[0]
num_classes = 7

labels = torch.tensor(nodes_df['label'].values, dtype=torch.long)
node_features = F.one_hot(labels, num_classes).float()
edge_index = torch.tensor(edges_df[['sourceNodeId', 'targetNodeId']].values.T, dtype=torch.long)

y = torch.tensor(homophily_df['homophily'].values, dtype=torch.float)
data = Data(x=node_features, edge_index=edge_index, y=y)

n_quantiles = 10
homophily_bins = pd.qcut(homophily_df['homophily'], q=n_quantiles, labels=False, duplicates='drop')

train_idx, test_idx = train_test_split(np.arange(num_nodes), test_size=0.2, random_state=42, stratify=homophily_bins)
train_idx, val_idx = train_test_split(train_idx, test_size=0.2, random_state=42, stratify=homophily_bins[train_idx])

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.dropout = 0.2
        self.lin = torch.nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lin(x).squeeze()
        return x

device = torch.device('cpu')
model = GCN(in_channels=num_classes, hidden_channels=64).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
patience = 15

best_val_mae = float('inf')
patience_counter = 0

for epoch in range(1, 300):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_pred = model(data.x, data.edge_index)[data.val_mask]
        val_true = data.y[data.val_mask]
        val_mae = mean_absolute_error(val_true.cpu(), val_pred.cpu())

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}, Train Loss: {loss.item():.4f}, Val MAE: {val_mae:.4f}")

    if val_mae < best_val_mae:
        best_val_mae = val_mae
        torch.save(model.state_dict(), 'best_model.pth')
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

model.load_state_dict(torch.load('best_model.pth'))
model.eval()
with torch.no_grad():
    test_pred = model(data.x, data.edge_index)[data.test_mask]
    test_true = data.y[data.test_mask]

    test_true_np = test_true.cpu().numpy()
    test_pred_np = test_pred.cpu().numpy()

    test_mae = mean_absolute_error(test_true_np, test_pred_np)
    mse = mean_squared_error(test_true_np, test_pred_np)
    r2 = r2_score(test_true_np, test_pred_np)
    mape = np.mean(np.abs((test_true_np - test_pred_np) / test_true_np)) * 100

print(f"\nTest MAE: {test_mae:.4f}")
print(f"Test MSE: {mse:.4f}")
print(f"Test R²: {r2:.4f}")
print(f"Mean Percentage Error: {mape:.2f}%")

Epoch 010, Train Loss: 0.0582, Val MAE: 0.1912
Epoch 020, Train Loss: 0.0254, Val MAE: 0.1105
Epoch 030, Train Loss: 0.0234, Val MAE: 0.1024
Epoch 040, Train Loss: 0.0196, Val MAE: 0.0901
Epoch 050, Train Loss: 0.0151, Val MAE: 0.0818
Epoch 060, Train Loss: 0.0125, Val MAE: 0.0720
Epoch 070, Train Loss: 0.0101, Val MAE: 0.0637
Epoch 080, Train Loss: 0.0082, Val MAE: 0.0555
Epoch 090, Train Loss: 0.0067, Val MAE: 0.0478
Epoch 100, Train Loss: 0.0054, Val MAE: 0.0402
Epoch 110, Train Loss: 0.0047, Val MAE: 0.0355
Epoch 120, Train Loss: 0.0042, Val MAE: 0.0329
Epoch 130, Train Loss: 0.0041, Val MAE: 0.0326
Epoch 140, Train Loss: 0.0039, Val MAE: 0.0323
Epoch 150, Train Loss: 0.0038, Val MAE: 0.0321
Epoch 160, Train Loss: 0.0038, Val MAE: 0.0320
Epoch 170, Train Loss: 0.0038, Val MAE: 0.0322
Epoch 180, Train Loss: 0.0036, Val MAE: 0.0321
Epoch 190, Train Loss: 0.0036, Val MAE: 0.0320
Early stopping at epoch 198
Test MAE: 0.0323
Test MSE: 0.0017
Test R²: 0.0914
Mean Percentage Error: 4.07%


In [26]:
# -----------------------------------------------------------------
# graph attention network implementation with synthetic graph data
# -----------------------------------------------------------------

num_nodes = nodes_df.shape[0]
assert homophily_df.shape[0] == num_nodes, "homophily.csv length must match nodes.csv length"

num_classes = 7
labels = torch.tensor(nodes_df['label'].values, dtype=torch.long)
x = F.one_hot(labels, num_classes).float()  # shape (N, 7)

edge_index = torch.tensor(edges_df[['sourceNodeId', 'targetNodeId']].values.T, dtype=torch.long)
rev_edge_index = edge_index.flip(0)
edge_index = torch.cat([edge_index, rev_edge_index], dim=1)

ei_np = edge_index.numpy()
ei_cols = np.ascontiguousarray(ei_np.T)
unique_cols = np.unique(ei_cols, axis=0)
edge_index = torch.tensor(unique_cols.T, dtype=torch.long)

y = torch.tensor(homophily_df['homophily'].values, dtype=torch.float)
data = Data(x=x, edge_index=edge_index, y=y)

n_quantiles = 10
homophily_bins = pd.qcut(homophily_df['homophily'], q=n_quantiles, labels=False, duplicates='drop')

all_idx = np.arange(num_nodes)
train_idx, test_idx = train_test_split(all_idx, test_size=0.2, random_state=42, stratify=homophily_bins)
train_idx, val_idx = train_test_split(train_idx, test_size=0.2, random_state=42, stratify=homophily_bins[train_idx])

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

class GATRegression(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels=64, heads=4, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.heads = heads

        assert hidden_channels % heads == 0, "hidden_channels must be divisible by heads"
        out_per_head = hidden_channels // heads

        self.gat1 = GATConv(in_channels, out_per_head, heads=heads, concat=True, dropout=dropout)
        self.gat2 = GATConv(hidden_channels, out_per_head, heads=heads, concat=True, dropout=dropout)

        self.lin = torch.nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.gat2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        out = self.lin(x).squeeze(-1)
        return out

device = torch.device('cpu')
model = GATRegression(in_channels=x.size(1), hidden_channels=64, heads=4, dropout=0.2).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
max_epochs = 500
patience = 15

best_val_mae = float('inf')
patience_counter = 0
best_path = 'best_model_gat.pth'

for epoch in range(1, max_epochs + 1):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_pred = model(data.x, data.edge_index)[data.val_mask]
        val_true = data.y[data.val_mask]
        val_mae = mean_absolute_error(val_true.cpu().numpy(), val_pred.cpu().numpy())

    if epoch % 5 == 0:
        print(f"Epoch {epoch:03d} | Train MSE: {loss.item():.6f} | Val MAE: {val_mae:.6f}")

    if val_mae < best_val_mae:
        best_val_mae = val_mae
        torch.save(model.state_dict(), best_path)
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}. Best val MAE: {best_val_mae:.6f}")
            break

model.load_state_dict(torch.load(best_path, map_location=device))
model.eval()
with torch.no_grad():
    test_pred = model(data.x, data.edge_index)[data.test_mask]
    test_true = data.y[data.test_mask]

    test_true_np = test_true.cpu().numpy()
    test_pred_np = test_pred.cpu().numpy()

    test_mae = mean_absolute_error(test_true_np, test_pred_np)
    test_mse = mean_squared_error(test_true_np, test_pred_np)
    test_r2 = r2_score(test_true_np, test_pred_np)

    nonzero_mask = test_true_np != 0
    if nonzero_mask.sum() > 0:
        mape = np.mean(np.abs((test_true_np[nonzero_mask] - test_pred_np[nonzero_mask]) / test_true_np[nonzero_mask])) * 100.
    else:
        mape = np.nan  # undefined if all true values are 0

print("\n--- GAT Results ---")
print(f"Test MAE: {test_mae:.6f}")
print(f"Test MSE: {test_mse:.6f}")
print(f"Test R²: {test_r2:.6f}")
print(f"Test MAPE: {mape if not np.isnan(mape) else 'NaN (all true=0)'}")


Epoch 005 | Train MSE: 0.015108 | Val MAE: 0.320632
Epoch 010 | Train MSE: 0.012707 | Val MAE: 0.105534
Epoch 015 | Train MSE: 0.060722 | Val MAE: 0.206206
Epoch 020 | Train MSE: 0.011602 | Val MAE: 0.088503
Epoch 025 | Train MSE: 0.020720 | Val MAE: 0.054455
Epoch 030 | Train MSE: 0.016916 | Val MAE: 0.104454
Epoch 035 | Train MSE: 0.010482 | Val MAE: 0.033787
Epoch 040 | Train MSE: 0.012685 | Val MAE: 0.049683
Epoch 045 | Train MSE: 0.009575 | Val MAE: 0.052089
Epoch 050 | Train MSE: 0.009677 | Val MAE: 0.035643
Epoch 055 | Train MSE: 0.009740 | Val MAE: 0.036698
Epoch 060 | Train MSE: 0.008841 | Val MAE: 0.039193
Epoch 065 | Train MSE: 0.008377 | Val MAE: 0.033461
Epoch 070 | Train MSE: 0.008516 | Val MAE: 0.032783
Epoch 075 | Train MSE: 0.008045 | Val MAE: 0.035647
Epoch 080 | Train MSE: 0.007944 | Val MAE: 0.032794
Epoch 085 | Train MSE: 0.007859 | Val MAE: 0.032946
Epoch 090 | Train MSE: 0.007641 | Val MAE: 0.033758
Epoch 095 | Train MSE: 0.007668 | Val MAE: 0.033033
Early stoppi

### PART 3 / 4:
CORA CITATION NETWORK AND FEATURE ENGINEERING

In [ ]:
# --------------------------------------------------
# prepare cora citation network dataset as NX Graph
# --------------------------------------------------

GRAPH_DIR = 'data/cora_graph'
OUT_FEATURE_DIR = 'data/cora_features'

os.makedirs('data/cora_graph', exist_ok=True)
nodes_url = "https://temprl.com/nodes.csv"
edges_url = "https://temprl.com/edges.csv"

def download_file(url, filename):
    response = requests.get(url)
    response.raise_for_status()
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {filename}")

download_file(nodes_url, "data/cora_graph/nodes.csv")
download_file(edges_url, "data/cora_graph/edges.csv")
nodes_df = pd.read_csv("data/cora_graph/nodes.csv")
edges_df = pd.read_csv("data/cora_graph/edges.csv")

G = nx.Graph()
G.add_nodes_from(nodes_df['nodeId'])
G.add_edges_from(zip(edges_df['sourceNodeId'], edges_df['targetNodeId']))
labels = nodes_df.set_index('nodeId')['subject'].to_dict()

homophily = {}
for node in tqdm(G.nodes):
    neighbors = list(G.neighbors(node))
    if not neighbors:
        homophily[node] = 0.0
        continue
    same_label_count = sum(labels.get(n, None) == labels.get(node, None) for n in neighbors)
    homophily[node] = same_label_count / len(neighbors)

homophily_df = pd.DataFrame(list(homophily.items()), columns=['nodeId', 'homophily'])
homophily_df.to_csv("data/cora_graph/homophily.csv", index=False)
print("Saved homophily.csv\n")

os.makedirs('data/cora_features', exist_ok=True)

feature_names = ['node_degree', 'local_triangle_density', 'average_neighbor_degree', 'eigenvector_centrality']
for feature in feature_names:
    df = pd.DataFrame({'nodeId': nodes_df['nodeId'], feature: 0})
    df.to_csv(f'data/cora_features/{feature}.csv', index=False)
print("Created data/cora_features folder and feature CSV placeholders.")

In [6]:
# ---------------------------------------------------
# Calculate and save features from NetworkX Cora data
# ---------------------------------------------------

nodes_df = pd.read_csv("data/cora_graph/nodes.csv")
edges_df = pd.read_csv("data/cora_graph/edges.csv")

node_degree = dict(G.degree())
local_triangle_density = nx.clustering(G)
avg_neighbor_degree = nx.average_neighbor_degree(G)
try:
    eigenvector_centrality = nx.eigenvector_centrality_numpy(G)
except Exception:
    eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)

features_df = pd.DataFrame({
    'nodeId': list(G.nodes()),
    'node_degree': [node_degree[n] for n in G.nodes()],
    'local_triangle_density': [local_triangle_density[n] for n in G.nodes()],
    'average_neighbor_degree': [avg_neighbor_degree[n] for n in G.nodes()],
    'eigenvector_centrality': [eigenvector_centrality[n] for n in G.nodes()]
})

for col in ['node_degree', 'local_triangle_density', 'average_neighbor_degree', 'eigenvector_centrality']:
    min_val = features_df[col].min()
    max_val = features_df[col].max()
    features_df[col] = (features_df[col] - min_val) / (max_val - min_val)

In [7]:
# ---------------------------------------------------
# Fit CORA data with linear regression implementation
# ---------------------------------------------------

homophily_df = pd.read_csv("data/cora_graph/homophily.csv")

data_df = pd.merge(features_df, homophily_df, on='nodeId')
data_df = data_df.dropna(subset=['homophily'])

X = data_df[['node_degree', 'local_triangle_density', 'average_neighbor_degree', 'eigenvector_centrality']]
y = data_df['homophily']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"Test MSE: {mse:.6f}")
print(f"Test R^2: {r2:.4f}")

def safe_mape(y_true, y_pred, epsilon=1e-10):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = np.abs(y_true) > epsilon
    if np.sum(mask) == 0: return np.inf
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

mape_val = safe_mape(y_test, y_pred) * 100
print(f"Test MAPE: {mape_val:.2f}%")

mean_pred = y_train.mean()
baseline_mape_val = safe_mape(y_test, np.full_like(y_test, mean_pred)) * 100
print(f"Baseline MAPE (predicting mean): {baseline_mape_val:.2f}%")

coef_df = pd.DataFrame({'feature': X.columns, 'coefficient': lr.coef_})
print(coef_df.sort_values(by='coefficient', ascending=False))

Test MSE: 0.084785
Test R^2: 0.0159
Test MAPE: 31.12%
Baseline MAPE (predicting mean): 31.97%
                   feature  coefficient
3   eigenvector_centrality     1.569000
1   local_triangle_density     0.129584
2  average_neighbor_degree    -0.250274
0              node_degree    -0.742211


In [ ]:
# ----------------------------------------------------
# Fit CORA data with graph convolutional network (gcn)
# ----------------------------------------------------


